In [ ]:
!pip install -q piper-tts pathvalidate soundfile librosa datasets transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 34.0 MB/s eta 0:00:00


In [ ]:
!python -m piper.download_voices en_US-lessac-medium

INFO:__main__:Downloaded: en_US-lessac-medium


In [ ]:
import os
import json
import subprocess
import torch
import soundfile as sf
import librosa
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoProcessor, Qwen2AudioForConditionalGeneration

# --------------- CONFIG ---------------

DATASET_ID = "theblackcat102/joke_explaination"
DATASET_SPLIT = "train"

OUT_PATH = "cache/joke_explanations_qwen_audio.jsonl"

MODEL_ID = "Qwen/Qwen2-Audio-7B-Instruct"
MAX_NEW_TOKENS = 120

# TTS
PIPER_MODEL = "/content/piper_models/en_US-lessac-medium.onnx"
AUDIO_DIR = "cache/joke_tts"
AUDIO_EXT = ".wav"

os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs("cache", exist_ok=True)

# ------------- PROMPT -------------

def build_messages(joke):
    return [
        {
            "role": "system",
            "content": "You are an expert linguist."
        },
        {
            "role": "user",
            "content": f"""Explain the following joke.

You are given the written joke and its spoken audio.

<Audio>
<|AUDIO|>
</Audio>

Instructions:
- Do NOT explain your analysis process.
- Focus ONLY on why the joke is humorous.
- Mention wordplay, ambiguity, or implied meaning if present.
- If the joke is not based on wordplay, explain the humor mechanism briefly.

Write a concise paragraph (3–6 sentences).

Joke:
{joke}
"""
        }
    ]

# -------------- HELPERS --------------

def normalize_id(idx):
    return f"joke_{idx}"

def load_audio(path, target_sr=16000):
    wav, sr = sf.read(path)
    if sr != target_sr:
        wav = librosa.resample(wav, orig_sr=sr, target_sr=target_sr)
    return wav

def generate_tts(text, uid):
    out_wav = os.path.join(AUDIO_DIR, uid + AUDIO_EXT)

    if os.path.exists(out_wav) and os.path.getsize(out_wav) > 1000:
        return True

    p = subprocess.run(
        [
            "piper",
            "--model", PIPER_MODEL,
            "--output_file", out_wav,
        ],
        input=text + "\n",
        text=True,
        capture_output=True,
    )

    return p.returncode == 0 and os.path.exists(out_wav)

# ---------------- MAIN ----------------

def main():
    if not torch.cuda.is_available():
        raise SystemExit("NO GPU DETECTED")

    device = "cuda"
    torch.set_grad_enabled(False)

    print(f"Loading model: {MODEL_ID}")
    processor = AutoProcessor.from_pretrained(MODEL_ID)
    model = Qwen2AudioForConditionalGeneration.from_pretrained(
        MODEL_ID,
        device_map="auto",
        torch_dtype=torch.float16,
    ).eval()

    print(f"Loading dataset: {DATASET_ID}")
    ds = load_dataset(DATASET_ID, split=DATASET_SPLIT)

    # ---- Phase A: TTS ----
    print("=== Generating TTS ===")
    for idx, item in tqdm(enumerate(ds), total=len(ds), desc="TTS"):
        joke = item.get("joke", "").strip()
        if not joke:
            continue
        uid = normalize_id(idx)
        generate_tts(joke, uid)

    # ---- Phase B: Inference ----
    def generate(joke, uid):
        messages = build_messages(joke)

        prompt = processor.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )

        audio = load_audio(os.path.join(AUDIO_DIR, uid + AUDIO_EXT))

        inputs = processor(
            text=prompt,
            audio=audio,
            sampling_rate=16000,
            return_tensors="pt",
            padding=True,
        ).to(device)

        with torch.no_grad():
            out = model.generate(
                **inputs,
                max_new_tokens=MAX_NEW_TOKENS,
                min_new_tokens=40,
                do_sample=False,
                pad_token_id=processor.tokenizer.eos_token_id,
            )

        gen_tokens = out[0][inputs["input_ids"].shape[1]:]
        return processor.tokenizer.decode(
            gen_tokens,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        ).strip()

    print("=== Explaining jokes (Text + Audio) ===")
    with open(OUT_PATH, "w", encoding="utf-8") as f:
        for idx, item in tqdm(enumerate(ds), total=len(ds), desc="Inference"):
            joke = item.get("joke", "").strip()
            if not joke:
                continue

            uid = normalize_id(idx)
            wav = os.path.join(AUDIO_DIR, uid + AUDIO_EXT)
            if not os.path.exists(wav):
                continue

            explanation = generate(joke, uid)

            out_obj = {
                "id": idx, 
                "Joke": joke,
                "Explanation": explanation,
                "URL": item.get("url"),
                "GoldExplanation": item.get("explanation"),
            }

            f.write(json.dumps(out_obj, ensure_ascii=False) + "\n")
            f.flush()

            torch.cuda.empty_cache()

    print("Done.")
    print(f"Output -> {OUT_PATH}")

# --------------- RUN ----------------

if __name__ == "__main__":
    main()


Loading model: Qwen/Qwen2-Audio-7B-Instruct


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/853 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

Loading dataset: theblackcat102/joke_explaination
=== Generating TTS ===



TTS: 100%|██████████| 377/377 [14:25<00:00,  2.30s/it]


=== Explaining jokes (Text + Audio) ===



Inference:   0%|          | 0/377 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.

Inference: 100%|██████████| 377/377 [36:24<00:00,  5.80s/it]

Done.
Output → cache/joke_explanations_qwen_audio.jsonl
